## Import core packages ----------------------------------------------------------

In [45]:
import chipwhisperer as cw
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tnrange, tqdm, tqdm_notebook
import numpy as np
import time
import matplotlib.pyplot as plt
from datetime import datetime
import os
from Crypto.Cipher import AES
plot_size = {'width':15, 'height':4}

## Setup scope --------------------------------------------------------------------

In [46]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET='TINYAES128C'
SS_VER='SS_VER_1_1'

In [ ]:
## Initialization routines for the CW scope
try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")

if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
elif "neorv32" in PLATFORM.lower():
    prog = cw.programmers.NEORV32Programmer
elif PLATFORM == "CW308_SAM4S":
    prog = cw.programmers.SAM4SProgrammer
else:
    prog = None

import time
time.sleep(0.05)
scope.default_setup()

if PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
    scope.io.target_pwr = 0
    time.sleep(0.2)
    scope.io.target_pwr = 1
    time.sleep(0.2)
def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1) #xmega needs more startup time
    elif "neorv32" in PLATFORM.lower():
        raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
    elif PLATFORM == "CW308_SAM4S":
        scope.io.nrst = 'low'
        time.sleep(0.25)
        scope.io.nrst = 'high_z'
        time.sleep(0.25)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)

## Download the firmware -------------------------------------------------------

In [ ]:
## Defined one env. variable with the location of the firmware (for convenience)
print(os.environ['CW_FIRMWARES'])
cw.program_target(scope, prog, os.path.join(os.environ['CW_FIRMWARES'], "simpleserial-aes","simpleserial-aes-{}.hex".format(PLATFORM)))

## Define demo helper functions ---------------------------------------------------

In [49]:
## Functions to compute correlation for simple side-channel analysis
##--
##
##---------------------------------------------------------------------------------------------------
def compute_mean_std_data_byte(np_traces, np_text_in, text_in_pos, number_traces):   
    # Create a StandardScaler for each byte position
    trace_mean_std  = StandardScaler()
    byte_scaler = StandardScaler()

    for i in tnrange(number_traces, desc='[INFO]: computing mean and std (byte pos: {})'.format(text_in_pos)):
        # partially fit the scaler of a byte position
        byte_scaler.partial_fit(np_text_in[i][text_in_pos].reshape(1, -1))
        trace_mean_std.partial_fit(np_traces[i].reshape(1, -1))
    
    return ([trace_mean_std.mean_, trace_mean_std.var_], [byte_scaler.mean_, byte_scaler.var_])

##--
##
##---------------------------------------------------------------------------------------------------
def compute_corr(np_traces, np_text_in, text_in_pos, number_traces):
    # Get group
    n_samples = np_traces.shape[1]
    samples_corr    = np.zeros(shape=(n_samples,), dtype=np.float64)

    all_means_std = compute_mean_std_data_byte(np_traces, np_text_in, text_in_pos, number_traces)
    samples_mean  = all_means_std[0][0]
    samples_std   = np.sqrt(all_means_std[0][1])
    metadata_mean = all_means_std[1][0]
    metadata_std  = np.sqrt(all_means_std[1][1])
    
    for i in tnrange(number_traces, desc='[INFO]: computing correlation (byte pos: {})'.format(text_in_pos)):
        samples_corr = np.add(samples_corr, (np_traces[i] - samples_mean) * (np_text_in[i][text_in_pos] - metadata_mean))

    if np.count_nonzero(metadata_std) == 0 or np.count_nonzero(samples_std) == 0:
        print ('[WARNING]: Metadata or samples standard deviation of AES Sbox plaintext {} and key {} is zero'.format(text_in_pos, 0))
        print ('[INFO]: Returning zero correlation')
    else:
        samples_corr = np.true_divide(samples_corr, ((number_traces - 1) * samples_std * metadata_std))
    return samples_corr

In [50]:
## Funtion to generate random input to encrypt with the crypto-algorithm
def get_next_plaintext():
    seed = int.from_bytes(os.urandom(4), byteorder="little")
    np.random.seed(seed)
    return np.random.bytes(16)

## To compute the inverse of the key scheduling

AES_Sbox = np.array([
        0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
        0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
        0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
        0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
        0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
        0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
        0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
        0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
        0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
        0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
        0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
        0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
        0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
        0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
        0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
        0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16
        ], dtype=np.uint8)

INV_SBOX = np.array([
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D
    ], dtype=np.uint8)

RCON = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)

def forward_key_schedule(key, n_rounds):
    round_keys = list(key)
    for i in range(4, 4*(n_rounds+1)):
        a0, a1, a2, a3 = round_keys[(i-1)*4 : i*4]
        if i % 4 == 0:
            a0, a1, a2, a3 = AES_Sbox[a1], AES_Sbox[a2], AES_Sbox[a3], AES_Sbox[a0]
            a0 = a0 ^ RCON[i//4]
        b0, b1, b2, b3 = round_keys[(i-4)*4 : (i-3)*4]
        round_keys.extend([a0^b0, a1^b1, a2^b2, a3^b3])
    return round_keys

def backward_key_schedule(last_round_key, n_rounds):
    round_keys = list(last_round_key)
    for i in range(n_rounds, 0, -1):
        b12 = round_keys[12] ^ round_keys[8]
        b13 = round_keys[13] ^ round_keys[9]
        b14 = round_keys[14] ^ round_keys[10]
        b15 = round_keys[15] ^ round_keys[11]

        b8  = round_keys[8 ] ^ round_keys[4]
        b9  = round_keys[9 ] ^ round_keys[5]
        b10 = round_keys[10] ^ round_keys[6]
        b11 = round_keys[11] ^ round_keys[7]

        b4 = round_keys[4] ^ round_keys[0]
        b5 = round_keys[5] ^ round_keys[1]
        b6 = round_keys[6] ^ round_keys[2]
        b7 = round_keys[7] ^ round_keys[3]

        a0, a1, a2, a3 = AES_Sbox[b13], AES_Sbox[b14], AES_Sbox[b15], AES_Sbox[b12]
        a0 = a0 ^ RCON[i]

        b0 = a0 ^ round_keys[0]
        b1 = a1 ^ round_keys[1]
        b2 = a2 ^ round_keys[2]
        b3 = a3 ^ round_keys[3]
        
        round_keys = [b0, b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13, b14, b15] + round_keys
    return round_keys

In [51]:
%%html 
<b>When VSCode dark mode active. Use this to render the progress-bars correcly </b>
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

### Set demo global variables -----------------------------------------------------------------

In [52]:
## Set the used key
key_as_string:str = "1199cc441199cc441199cc441199cc44"
assert len(key_as_string) == 32, "Incorrect key lenght: {}".format(len(key_as_string))
AES_KEY = bytearray.fromhex(key_as_string)
## Number of traces to collect
NUMBER_OF_TRACES_TO_COLLECT = 2500
## PC AES algorithm instance
AES128_PC_Instance = AES.new(AES_KEY, AES.MODE_ECB)

############################# VARIABLES FOR THE ACQUISITION CAMPAING ######################################
enc_trace_array:list       = []
enc_textin_array:list      = []
enc_textout_array:list     = []
enc_dev_textout_array:list = []

dec_trace_array:list       = []
dec_textin_array:list      = []
dec_textout_array:list     = []
dec_dev_textout_array:list = []

## for printing purpose (not need to touch it)
np.set_printoptions(formatter={'int':lambda x:hex(int(x))[2:]})

##### Test the acquisition setup comparing device-PC encryption

In [ ]:
enc_trace_array:list       = []
enc_textin_array:list      = []
enc_textout_array:list     = []
enc_dev_textout_array:list = []
## -------------------------------------------------------------------------------
number_of_samples = 5
for i in tnrange(number_of_samples, desc='[INFO]: Generating inputs for encryption: '):
    text = get_next_plaintext()
    ciphertext = AES128_PC_Instance.encrypt(text)
    enc_textout_array.append(np.frombuffer(ciphertext, dtype=np.uint8))
    enc_textin_array.append(np.frombuffer(text, dtype=np.uint8))
## -------------------------------------------------------------------------------
target.set_key(AES_KEY) ## set the key in the target
for i in tnrange(number_of_samples, desc='[INFO]: Capturing traces: '):
    scope.arm()
    text = enc_textin_array[i].tobytes()
    # Sending the plaintext
    target.simpleserial_write('p', text)
    # Capturing the power traces
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue  
    # Read the ciphertext back
    response = target.simpleserial_read('r', 16)
    # Store the trace
    enc_trace_array.append(scope.get_last_trace())
    enc_dev_textout_array.append(np.frombuffer(response, dtype=np.uint8))
    
for i in tnrange(number_of_samples, desc='[INFO]: Asserting: '):
    if not ((enc_dev_textout_array[i]==enc_textout_array[i]).all()):
        print ("Warning, found not equal outputs")

plt.style.use('./pltstyle.mplstyle')
plt.plot(enc_trace_array[0])
plt.show()
plt.close()

##### Test the acquisition setup comparing device-PC decryption

In [ ]:
dec_trace_array:list       = []
dec_textin_array:list      = []
dec_textout_array:list     = []
dec_dev_textout_array:list = []
## -------------------------------------------------------------------------------
number_of_samples = 5
for i in tnrange(number_of_samples, desc='[INFO]: Generating inputs for decryption: '):
    text = get_next_plaintext()
    plaintext = AES128_PC_Instance.decrypt(text)
    dec_textout_array.append(np.frombuffer(plaintext, dtype=np.uint8))
    dec_textin_array.append(np.frombuffer(text, dtype=np.uint8))
## -------------------------------------------------------------------------------
target.set_key(AES_KEY) ## set the key in the target
for i in tnrange(number_of_samples, desc='[INFO]: Capturing traces: '):
    scope.arm()
    text = dec_textin_array[i].tobytes()
    # Sending the plaintext
    target.simpleserial_write('c', text)
    # Capturing the power traces
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue  
    # Read the plaintext back
    response = target.simpleserial_read('r', 16)
    # Store the trace
    dec_trace_array.append(scope.get_last_trace())
    dec_dev_textout_array.append(np.frombuffer(response, dtype=np.uint8))
    
for i in tnrange(number_of_samples, desc='[INFO]: Asserting: '):
    if not ((dec_dev_textout_array[i]==dec_textout_array[i]).all()):
        print ("Warning, found not equal outputs")

plt.style.use('./pltstyle.mplstyle')
plt.plot(dec_trace_array[0])
plt.show()
plt.close()

## Performing acquisition campaing ---------------------------------------

### 0. Generating input for the campaing

In [ ]:
enc_trace_array:list       = []
enc_textin_array:list      = []
enc_textout_array:list     = []
enc_dev_textout_array:list = []

dec_trace_array:list       = []
dec_textin_array:list      = []
dec_textout_array:list     = []
dec_dev_textout_array:list = []

## -------------------------------------------------------------------------------
for i in tnrange(NUMBER_OF_TRACES_TO_COLLECT, desc='[INFO]: Generating inputs for encryption: '):
    text = get_next_plaintext()
    ciphertext = AES128_PC_Instance.encrypt(text)
    enc_textout_array.append(np.frombuffer(ciphertext, dtype=np.uint8))
    enc_textin_array.append(np.frombuffer(text, dtype=np.uint8))
## -------------------------------------------------------------------------------
for i in tnrange(NUMBER_OF_TRACES_TO_COLLECT, desc='[INFO]: Generating inputs for decryption: '):
    text = enc_textout_array[i]
    plaintext = AES128_PC_Instance.decrypt(text.tobytes())
    dec_textout_array.append(np.frombuffer(plaintext, dtype=np.uint8))
    dec_textin_array.append(np.frombuffer(text, dtype=np.uint8))

### 1. Capturing traces for the encryption task

In [ ]:
target.set_key(AES_KEY) ## set the key in the target
for i in tnrange(NUMBER_OF_TRACES_TO_COLLECT, desc='[INFO]: Capturing traces: '):
    scope.arm()
    text = enc_textin_array[i].tobytes()
    # Sending the plaintext
    target.simpleserial_write('p', text)
    # Capturing the power traces
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    # Read the ciphertext back
    response = target.simpleserial_read('r', 16)
    # Store the trace
    enc_trace_array.append(scope.get_last_trace())
    enc_dev_textout_array.append(np.frombuffer(response, dtype=np.uint8))

for i in tnrange(NUMBER_OF_TRACES_TO_COLLECT, desc='[INFO]: Asserting: '):
    if not ((enc_dev_textout_array[i]==enc_textout_array[i]).all()):
        print ("Warning, found not equal outputs")

### 2. Capturing traces for the decryption task

In [ ]:
target.set_key(AES_KEY) ## set the key in the target
for i in tnrange(NUMBER_OF_TRACES_TO_COLLECT, desc='[INFO]: Capturing traces: '):
    scope.arm()
    text = dec_textin_array[i].tobytes()
    # Sending the plaintext
    target.simpleserial_write('c', text)
    # Capturing the power traces
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue  
    # Read the plaintext back
    response = target.simpleserial_read('r', 16)
    # Store the trace
    dec_trace_array.append(scope.get_last_trace())
    dec_dev_textout_array.append(np.frombuffer(response, dtype=np.uint8))
    
for i in tnrange(NUMBER_OF_TRACES_TO_COLLECT, desc='[INFO]: Asserting: '):
    if not ((dec_dev_textout_array[i]==dec_textout_array[i]).all()):
        print ("Warning, found not equal outputs")

## Summary ---------------------------------------------------------------------

In [58]:
enc_trace_array_np       = np.array(enc_trace_array)
enc_textin_array_np      = np.array(enc_textin_array)
enc_textout_array_np     = np.array(enc_textout_array)
enc_dev_textout_array_np = np.array(enc_dev_textout_array)
dec_trace_array_np       = np.array(dec_trace_array)
dec_textin_array_np      = np.array(dec_textin_array)
dec_textout_array_np     = np.array(dec_textout_array)
dec_dev_textout_array_np = np.array(dec_dev_textout_array)

print ("PC encryption:")
print ("Plaintext shape:", enc_textin_array_np.shape)
print ("Plaintext [0]:", enc_textin_array_np[0])
print ("Plaintext [1]:", enc_textin_array_np[1])
print ("Plaintext [2]:", enc_textin_array_np[2])
print ("DEVICE encryption:")
print ("Plaintext shape:", dec_dev_textout_array_np.shape)
print ("Plaintext [0]:", dec_dev_textout_array_np[0])
print ("Plaintext [1]:", dec_dev_textout_array_np[1])
print ("Plaintext [2]:", dec_dev_textout_array_np[2])

print ("PC decryption:")
print ("Ciphertext shape:", dec_textin_array_np.shape)
print ("Ciphertext [0]:", dec_textin_array_np[0])
print ("Ciphertext [1]:", dec_textin_array_np[1])
print ("Ciphertext [2]:", dec_textin_array_np[2])
print ("DEVICE decryption:")
print ("Ciphertext shape:", enc_dev_textout_array_np.shape)
print ("Ciphertext [0]:", enc_dev_textout_array_np[0])
print ("Ciphertext [1]:", enc_dev_textout_array_np[1])
print ("Ciphertext [2]:", enc_dev_textout_array_np[2])

print ("Encryption trace:")
print ("enc_trace_array_np", enc_trace_array_np.shape)

print ("Decryption trace:")
print ("dec_trace_array_np", dec_trace_array_np.shape)

PC encryption:
Plaintext shape: (2500, 16)
Plaintext [0]: [c9 b6 8e 70 f0 df 8f c1 64 26 92 67 71 3b ba b4]
Plaintext [1]: [4 d d0 90 18 5f 7b fb 8d ab f6 58 55 32 87 26]
Plaintext [2]: [e1 e4 33 ff e3 6c 9d bc 30 fd 8c 57 9f 32 97 82]
DEVICE encryption:
Plaintext shape: (2500, 16)
Plaintext [0]: [c9 b6 8e 70 f0 df 8f c1 64 26 92 67 71 3b ba b4]
Plaintext [1]: [4 d d0 90 18 5f 7b fb 8d ab f6 58 55 32 87 26]
Plaintext [2]: [e1 e4 33 ff e3 6c 9d bc 30 fd 8c 57 9f 32 97 82]
PC decryption:
Ciphertext shape: (2500, 16)
Ciphertext [0]: [81 d0 4d 93 ab 90 49 ba 74 e3 84 c6 c6 64 be dd]
Ciphertext [1]: [c5 ea 9f cf 4c d4 b5 86 58 42 75 94 c4 73 fe 31]
Ciphertext [2]: [94 99 b a4 e2 bc 54 b1 8f 5a 22 fa fa a4 a6 c1]
DEVICE decryption:
Ciphertext shape: (2500, 16)
Ciphertext [0]: [81 d0 4d 93 ab 90 49 ba 74 e3 84 c6 c6 64 be dd]
Ciphertext [1]: [c5 ea 9f cf 4c d4 b5 86 58 42 75 94 c4 73 fe 31]
Ciphertext [2]: [94 99 b a4 e2 bc 54 b1 8f 5a 22 fa fa a4 a6 c1]
Encryption trace:
enc_trace_array_np (

### 1.2 Store the side-channel information

In [59]:
np.save("enc_trace_array_np.npy", enc_trace_array_np)
np.save("enc_textin_array_np.npy", enc_textin_array_np)
np.save("enc_textout_array_np.npy", enc_textout_array_np)
np.save("enc_dev_textout_array_np.npy", enc_dev_textout_array_np)
np.save("dec_trace_array_np.npy", dec_trace_array_np)
np.save("dec_textin_array_np.npy", dec_textin_array_np)
np.save("dec_textout_array_np.npy", dec_textout_array_np)
np.save("dec_dev_textout_array_np.npy", dec_dev_textout_array_np)

## A bit of Side-channel analysis

In [ ]:
corr = compute_corr(enc_trace_array_np, enc_textout_array_np, 10, NUMBER_OF_TRACES_TO_COLLECT)
plt.plot(abs(corr))
plt.show()

In [ ]:
corr = compute_corr(dec_trace_array_np, dec_textin_array_np, 0, NUMBER_OF_TRACES_TO_COLLECT)
plt.plot(abs(corr))
plt.show()

## Performing the side-channel attack

In [ ]:
databyte_pos_init = 0
databyte_pos_end  = 16
data_length       = 16 #por AES-128
used_points_init  = 0
used_points_end   = dec_trace_array_np.shape[1]
verbose           = True
number_of_bits    = 8
trs_dataset       = dec_trace_array_np
ciphertext_array  = dec_textin_array_np
interval = [0, ciphertext_array.shape[0]]

for bit in tnrange(number_of_bits, desc="[INFO]: Per bit iteration", leave=True):
    recovered_key = []
    for byte_pos in tnrange(data_length, desc="[INFO]: Per byte iteration", leave=False):
        delta = []
        for key_guess in tnrange(256, desc="[INFO]: Per byte guessing", leave=False):
            zero_count = 0
            one_count  = 0
            zero_list  = np.array([0.0] * (used_points_end-used_points_init))
            one_list   = np.array([0.0] * (used_points_end-used_points_init))
            
            #-------------------------------------------------------------------------------------------
            # On of the interesting parts --------------------------------------------------------
            for trace_index in range (interval[0], interval[1]):
                intermediate_value = INV_SBOX[ciphertext_array[trace_index][byte_pos] ^ key_guess]
                target_bit = (intermediate_value >> bit) & 1
                if target_bit == 0:
                    zero_list  += trs_dataset[trace_index][used_points_init:used_points_end]
                    zero_count += 1
                else:
                    one_list   += trs_dataset[trace_index][used_points_init:used_points_end]
                    one_count  += 1
                    
            #-------------------------------------------------------------------------------------------
            
            # Here you compute the means of each group (group 1 and group 0)
            mean_delta_accu = np.abs((one_list/one_count) - (zero_list/zero_count))
            delta.append(np.max(mean_delta_accu))
        
        assert len(delta) == 256
        delta = np.array(delta)
        predicted_byte = delta.argmax()
        recovered_key.append(predicted_byte)
    print("Round key:", bytes(recovered_key[:16]).hex())
    round_keys = backward_key_schedule(recovered_key, n_rounds=10)
    print("Possible key:", bytes(round_keys[:16]).hex())

## Testing the found keys -------------------------------------------------------------

In [64]:
FOUND_KEY_string:str = "1199cc441199cc441199cc441199cc44"
FOUND_KEY = bytearray.fromhex(FOUND_KEY_string)
aes128 = AES.new(FOUND_KEY, AES.MODE_ECB)

plaintext = enc_textin_array_np[0].tobytes()
ciphertext = aes128.encrypt(plaintext)
if not (enc_textout_array_np[0] == np.frombuffer(ciphertext, dtype=np.uint8)).all():
    print("[INFO]: You shall no PASS!!")
else:
    print("[INFO]: Run fools (º_º)!!")

[INFO]: Run fools (º_º)!!
